# COMP0005 - GROUP COURSEWORK 2023-24
# Gesture Recognition via Convex Hull 

Use the cell below for all python code needed to realise the **Jarvis march algorithm** (including auxiliary data structures and functions needed by this algorithm - if any). The `jarvismarch()` function itself should take as input parameter a list of 2D points (`inputSet`), and return the subset of such points (`outputSet`) that lie on the convex hull.

In [1]:
import math
def orientation(p1, p2, p3): # Orientation works by checking the cross product of the vectors p1p2 and p2p3. 
    return (p3[1]-p2[1])*(p2[0]-p1[0]) - (p2[1]-p1[1])*(p3[0]-p2[0])

def leftmost(points: list) -> tuple: #Finds the leftmost point (lowest x-coord)
    left = points[0]
    for point in points:
        if point[0] < left[0]:
            left = point
    return left

def jarvismarch(inputSet): 
    left = leftmost(inputSet)
    onHull = left #leftmost point is starting point 
    outputSet = [] 
    nextPoint = None
    while nextPoint !=left:
        outputSet.append(onHull) 
        nextPoint = inputSet[0] 
        for point in inputSet:
            if orientation(onHull, nextPoint, point) > 0 or nextPoint == onHull: 
                nextPoint = point
        onHull = nextPoint
    return outputSet

import random
import timeit
examplePoints =  [(random.uniform(0, 100), random.uniform(0, 100)) for _ in range(200000)]
execution_time = timeit.timeit(lambda: jarvismarch(examplePoints), number=1)
print(f"Execution time: {execution_time:.6f} seconds")

Use the cell below for all python code needed to realise the **Graham scan** algorithm (including auxiliary data structures and functions needed by this algorithm - if any). The `grahamscan()` function itself should take as input parameter a list of 2D points (`inputSet`), and return the subset of such points that lie on the convex hull (`outputSet`).

In [17]:
# Hello
# Graham Scan
import math

def findLowestPoint(inputSet):
   lowestPoint = inputSet[0]  
   lowestIndex = 0
   for i in range(len(inputSet)):
      # Choose lowest y, break ties by choosing lowest x
      if inputSet[i][1] < lowestPoint[1] or (inputSet[i][1] == lowestPoint[1] and inputSet[i][0] < lowestPoint[0]):
         lowestPoint = inputSet[i]
         lowestIndex = i
 
   return lowestIndex, lowestPoint

def turnMade(point1, point2, point3):
    """Parameters:
    point1, point2, point3 (tuple): The points as tuples of x and y coordinates.

    Returns:
    float: The cross product of the vectors formed by point1-point2 and point1-point3.

    Determine the relative direction of the turn made by three points.

    The function calculates the cross product of the vectors formed by point1-point2 and point1-point3. 
    The sign of the result indicates the direction of the turn:
    - If the result is positive, the direction is counter-clockwise.
    - If the result is negative, the direction is clockwise.
    - If the result is zero, the points are collinear."""
   
    return (point2[0] - point1[0]) * (point3[1]-point1[1]) - (point2[1]-point1[1]) * (point3[0] - point1[0])


def polarAngle(point1, point2):
    """Parameters:
    point1 (tuple): The origin point as a tuple of x and y coordinates.
    point2 (tuple): The second point as a tuple of x and y coordinates.

    Returns:
    float: The polar angle in radians.

    Calculate the polar angle between two points.

    The polar angle is the counterclockwise angle in radians from the x-axis 
    to the vector pointing from the origin to the point. In this case, the origin 
    is `point1` and the point is `point2`."""

    return math.atan2((point2[1] - point1[1]), (point2[0] - point1[0]))


def grahamscan(inputSet):
    
    """Perform the Graham scan algorithm to find the convex hull of a set of points.

    1.) The function first finds the point with the lowest y-coordinate, breaks ties by going for lowest x-coordinate 

    2.) It swaps it with the first item in the inputSet

    3.) The function then sorts the remaining points based on their polar angle 
    with the lowest point, breaking ties by distance from the lowest point. 

    4.) The sorted points are then processed from beginning to end. 
    5.) For each point, while there is a right turn (clockwise direction) made by the path (considering the 3 most recent points)
        6.) the last point is removed from the convex hull
        7.) Else: The current point is then added to the convex hull.

    Parameters:
    inputSet (list): A list of 2D points as tuples of x and y coordinates.

    Returns:
    outputSet (list): A list of 2D points that lie on the convex hull."""
    # 1 2
    lowestIndex, lowestPoint = findLowestPoint(inputSet) 

    inputSet[0] , inputSet[lowestIndex] = inputSet[lowestIndex] , inputSet[0]

    # 3
    sortedPoints = sorted(inputSet[1:], key=lambda point: polarAngle(lowestPoint, point))

    convexHull = []

    # LowestPoint
    convexHull.append(inputSet[0])
    # First connection to lowestPoint
    convexHull.append(sortedPoints[0])

    i = 2
    previousAngleWasColinear = False
    # 4
    while i < len(sortedPoints):
        lastPoint = convexHull[-1]
        lastPointBefore = convexHull[-2]
        pointConsidered = sortedPoints[i]

        turnDirection = turnMade(lastPointBefore, lastPoint, pointConsidered)

        # Counter-Clockwise of the most recent three points, so add to the hull
        # 7
        if (turnDirection > 0):
            convexHull.append(pointConsidered)
            previousAngleWasColinear = False
            i += 1
        # 5 6
        # Clockwise of the most recent three points, so pop from the hull
        elif len(convexHull) > 2 and turnDirection < 0:
            convexHull.pop()
            previousAngleWasColinear = False
        elif (turnDirection == 0):
            # If we're in a sequence of colinear points, pop the previous node because we've found another colinear point
            # Or edge case: where the third point (2nd index) in sortedPoints started a colinear sequence, so the second point (1st index) should be discarded
            if previousAngleWasColinear == True or i == 2:
                convexHull.pop()
            convexHull.append(pointConsidered)
            previousAngleWasColinear = True
            i += 1
        else:
            i+=1
                    
            
    outputSet = convexHull
    return outputSet


#print(grahamscan(inputSet))


import random
import timeit

resolution = 100
"""
# Input is average case (random points)
inputSet = [(random.uniform(0, 1000), random.uniform(0, 1000)) for _ in range(resolution)]
"""
### Input is the same line
slope = 3#random.uniform(-10, 10)
intercept = 0#random.uniform(-10, 10)
# Generate 100000 collinear points along the line
inputSet = [(x, slope * x + intercept) for x in range(resolution)]
print(inputSet)
"""
### Input is a circle
# Choose a center and a radius for the circle
center = (random.uniform(-10, 10), random.uniform(-10, 10))
radius = random.uniform(1, 10)
# Generate 1000 points along the circumference of the circle
inputSet = [(center[0] + radius * math.cos(2 * math.pi * i / 1000), center[1] + radius * math.sin(2 * math.pi * i / 1000)) for i in range(1000)]

### Input is a Triangle
# Choose three vertices for the triangle
vertices = [(0, 0), (1, 0), (0.5, math.sqrt(3)/2)]
# Generate points along the edges of the triangle
inputSet = []
# Edge from vertices[0] to vertices[1]
for t in range(resolution):
    x = vertices[0][0] + t * (vertices[1][0] - vertices[0][0]) / resolution
    y = vertices[0][1] + t * (vertices[1][1] - vertices[0][1]) / resolution
    inputSet.append((x, y))
# Edge from vertices[1] to vertices[2]
for t in range(resolution):
    x = vertices[1][0] + t * (vertices[2][0] - vertices[1][0]) / resolution
    y = vertices[1][1] + t * (vertices[2][1] - vertices[1][1]) / resolution
    inputSet.append((x, y))
# Edge from vertices[2] to vertices[0]
for t in range(resolution):
    x = vertices[2][0] + t * (vertices[0][0] - vertices[2][0]) / resolution
    y = vertices[2][1] + t * (vertices[0][1] - vertices[2][1]) / resolution
    inputSet.append((x, y))
"""
# Wrap your function call in a timeit statement
execution_time = timeit.timeit(lambda: grahamscan(inputSet), number=1)

print(f"Execution time: {execution_time:.6f} seconds")

print(grahamscan(inputSet))


[(0, 0), (1, 3), (2, 6), (3, 9), (4, 12), (5, 15), (6, 18), (7, 21), (8, 24), (9, 27), (10, 30), (11, 33), (12, 36), (13, 39), (14, 42), (15, 45), (16, 48), (17, 51), (18, 54), (19, 57), (20, 60), (21, 63), (22, 66), (23, 69), (24, 72), (25, 75), (26, 78), (27, 81), (28, 84), (29, 87), (30, 90), (31, 93), (32, 96), (33, 99), (34, 102), (35, 105), (36, 108), (37, 111), (38, 114), (39, 117), (40, 120), (41, 123), (42, 126), (43, 129), (44, 132), (45, 135), (46, 138), (47, 141), (48, 144), (49, 147), (50, 150), (51, 153), (52, 156), (53, 159), (54, 162), (55, 165), (56, 168), (57, 171), (58, 174), (59, 177), (60, 180), (61, 183), (62, 186), (63, 189), (64, 192), (65, 195), (66, 198), (67, 201), (68, 204), (69, 207), (70, 210), (71, 213), (72, 216), (73, 219), (74, 222), (75, 225), (76, 228), (77, 231), (78, 234), (79, 237), (80, 240), (81, 243), (82, 246), (83, 249), (84, 252), (85, 255), (86, 258), (87, 261), (88, 264), (89, 267), (90, 270), (91, 273), (92, 276), (93, 279), (94, 282), (9

Use the cell below for all python code needed to realise the **Chen's** algorithm (including auxiliary data structures and functions needed by this algorithm - if any). The `chen()` function itself should take as input parameter a list of 2D points (`inputSet`), and return the subset of such points that lie on the convex hull (`outputSet`).

In [3]:
def chen(inputSet):
    '''
    Returns the list of points that lie on the convex hull (chen's algorithm)
            Parameters:
                    inputSet (list): a list of 2D points

            Returns:
                    outputSet (list): a list of 2D points
    '''

    #ADD YOUR CODE HERE


    return outputSet

Use the cell below to implement the **synthetic data generator** needed by your experimental framework (including any auxiliary data structures and functions you might need - be mindful of code readability and reusability).

In [4]:
import random

class TestDataGenerator():
    """
    A class to represent a synthetic data generator.

    ...

    Attributes
    ----------
    
    [to be defined as part of the coursework]

    Methods
    -------
    
    [to be defined as part of the coursework]

    """
        
    #ADD YOUR CODE HERE
    
    def __init__():
        pass


Use the cell below to implement the requested **experimental framework** API.

In [5]:
import timeit
import matplotlib

class ExperimentalFramework():
    """
    A class to represent an experimental framework.

    ...

    Attributes
    ----------
    
    [to be defined as part of the coursework]

    Methods
    -------
    
    [to be defined as part of the coursework]

    """
        
    #ADD YOUR CODE HERE
    
    def __init__():
        pass

Use the cell below to illustrate the python code you used to **fully evaluate** the three convex hull algortihms under considerations. The code below should illustrate, for example, how you made used of the **TestDataGenerator** class to generate test data of various size and properties; how you instatiated the **ExperimentalFramework** class to  evaluate each algorithm using such data, collect information about their execution time, plots results, etc. Any results you illustrate in the companion PDF report should have been generated using the code below.

In [6]:
# ADD YOUR TEST CODE HERE 



